# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import timedelta
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Format today and yesterday into strings for filename creation
format = "%m-%d-%Y"
now = datetime.now()
today = now.strftime(format)
yesterday = (now - timedelta(days=1)).strftime(format)

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    jh_live_date = today
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    jh_live_date = yesterday
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
print(jh_live_date)

09-06-2022


In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2022-09-07 04:20:59,33.93911,67.709953,194614,7783,NaN,NaN,Afghanistan,499.928831,3.999198
1,NaN,NaN,NaN,Albania,2022-09-07 04:20:59,41.15330,20.168300,330283,3583,NaN,NaN,Albania,11476.926819,1.084827
2,NaN,NaN,NaN,Algeria,2022-09-07 04:20:59,28.03390,1.659600,270476,6879,NaN,NaN,Algeria,616.806309,2.543294
3,NaN,NaN,NaN,Andorra,2022-09-07 04:20:59,42.50630,1.521800,46027,154,NaN,NaN,Andorra,59570.309972,0.334586
4,NaN,NaN,NaN,Angola,2022-09-07 04:20:59,-11.20270,17.873900,102636,1917,NaN,NaN,Angola,312.283707,1.867766


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/28/22,8/29/22,8/30/22,8/31/22,9/1/22,9/2/22,9/3/22,9/4/22,9/5/22,9/6/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,192463,192906,193004,193250,193520,193520,193912,194163,194355,194614
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,328515,328571,329017,329352,329615,329862,330062,330193,330221,330283
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,270194,270235,270272,270304,270359,270405,270426,270443,270461,270476


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/28/22,8/29/22,8/30/22,8/31/22,9/1/22,9/2/22,9/3/22,9/4/22,9/5/22,9/6/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7777,7777,7777,7777,7778,7778,7780,7782,7783,7783
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3581,3581,3582,3582,3583,3583,3583,3583,3583,3583
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6878,6878,6878,6878,6878,6878,6878,6879,6879,6879


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/28/22,8/29/22,8/30/22,8/31/22,9/1/22,9/2/22,9/3/22,9/4/22,9/5/22,9/6/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/28/22,8/29/22,8/30/22,8/31/22,9/1/22,9/2/22,9/3/22,9/4/22,9/5/22,9/6/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,17991,17991,17991,17991,18125,18125,18125,18125,18125,18125
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,64095,64095,64095,64095,64672,64672,64672,64672,64672,64672
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,6650,6650,6650,6650,6757,6757,6757,6757,6757,6757


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/28/22,8/29/22,8/30/22,8/31/22,9/1/22,9/2/22,9/3/22,9/4/22,9/5/22,9/6/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,222,222,222,222,223,223,223,223,223,223
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,693,693,693,693,699,699,699,699,699,699
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,101,101,101,101,101,101,101,101,101,101


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
675,1001.0,Autauga,Alabama,US,2022-09-07 04:20:59,32.539527,-86.644082,18125,223,NaN,NaN,"Autauga, Alabama, US",32441.962448,1.230345
712,1075.0,Lamar,Alabama,US,2022-09-07 04:20:59,33.779950,-88.096680,4303,68,NaN,NaN,"Lamar, Alabama, US",31169.865991,1.580293
713,1077.0,Lauderdale,Alabama,US,2022-09-07 04:20:59,34.901719,-87.656247,27767,405,NaN,NaN,"Lauderdale, Alabama, US",29944.246137,1.458566


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,94880701,1048186,0.0
India,44464284,528030,0.0
France,34839791,155373,0.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,94880701,1048186,0.0,93832515.0,2022-09-07 04:20:59,37.934269,-91.444083
India,44464284,528030,0.0,43936254.0,2022-09-07 04:20:59,23.088275,81.806127
France,34839791,155373,0.0,34684418.0,2022-09-07 04:20:59,6.430808,-34.730285
Brazil,34477539,684425,0.0,33793114.0,2022-09-07 04:20:59,-12.669522,-48.480493
Germany,32344032,147981,0.0,32196051.0,2022-09-07 04:20:59,51.571844,10.277881


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,11127471,95072,0.0
Texas,7892204,90415,0.0
Florida,7051414,80231,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,11127471,95072,0.0,11032399.0,2022-09-07 04:20:59,37.843962,-120.728594
Texas,7892204,90415,0.0,7801789.0,2022-09-07 04:20:59,31.660643,-98.653069
Florida,7051414,80231,0.0,6971183.0,2022-09-07 04:20:59,28.940755,-82.700744
New York,6012162,71222,0.0,5940940.0,2022-09-07 04:20:59,42.544151,-75.474183
Illinois,3706263,39381,0.0,3666882.0,2022-09-07 04:20:59,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,3420213,33217,0.0
Florida,Miami-Dade,1438468,11614,0.0
Arizona,Maricopa,1415836,17792,0.0
Illinois,Cook,1385187,14725,0.0
Texas,Harris,1202227,11248,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,3420213,33217,0.0,3386996.0,2022-09-07 04:20:59,34.308284,-118.228241,6037.0
Florida,Miami-Dade,1438468,11614,0.0,1426854.0,2022-09-07 04:20:59,25.611236,-80.551706,12086.0
Arizona,Maricopa,1415836,17792,0.0,1398044.0,2022-09-07 04:20:59,33.348359,-112.491815,4013.0
Illinois,Cook,1385187,14725,0.0,1370462.0,2022-09-07 04:20:59,41.841448,-87.816588,17031.0
Texas,Harris,1202227,11248,0.0,1190979.0,2022-09-07 04:20:59,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-04,194163,330193,270443,46027,102636,11,8974,9689861,434398,10072887,...,979160,243830,11836,542874,11418894,702332,535,11931,333074,256763
2022-09-05,194355,330221,270461,46027,102636,11,8974,9689861,436727,10083201,...,981186,243863,11864,542952,11421055,702332,535,11931,333086,256769
2022-09-06,194614,330283,270476,46027,102636,11,8974,9689861,436727,10095000,...,981186,243879,11892,543123,11424754,702591,535,11931,333124,256782


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-04,7782,3583,6879,154,1917,0,145,129769,8655,14077,...,7445,1637,14,5798,43120,5704,0,2155,4017,5596
2022-09-05,7783,3583,6879,154,1917,0,145,129769,8662,14145,...,7455,1637,14,5801,43122,5704,0,2155,4017,5596
2022-09-06,7783,3583,6879,154,1917,0,145,129769,8662,14214,...,7455,1637,14,5803,43123,5706,0,2155,4017,5596


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-09-05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-09-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/28/22,8/29/22,8/30/22,8/31/22,9/1/22,9/2/22,9/3/22,9/4/22,9/5/22,9/6/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,17991,17991,17991,17991,18125,18125,18125,18125,18125,18125
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,4249,4249,4249,4249,4303,4303,4303,4303,4303,4303
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,27464,27464,27464,27464,27767,27767,27767,27767,27767,27767
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,8590,8590,8590,8590,8700,8700,8700,8700,8700,8700
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,43115,43115,43115,43115,43516,43516,43516,43516,43516,43516


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-09-04,1494300,294498,8168,2254374,936411,11120267,1634692,877823,303196,49,...,2302525,7869972,1029759,140308,22771,2045387,1787212,584771,1835416,174828
2022-09-05,1494300,294498,8168,2254374,936927,11125282,1636347,878056,303196,49,...,2302525,7870131,1029759,140308,22771,2050814,1787212,584771,1838108,174828
2022-09-06,1494300,294498,8168,2254374,937356,11127471,1636657,878267,303788,49,...,2309566,7892204,1029759,140308,22844,2052404,1787212,587334,1838770,175290


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2022-09-04       18125   64672    6757  7394  16597    2750   5920   36113   
2022-09-05       18125   64672    6757  7394  16597    2750   5920   36113   
2022-09-06       18125   64672    6757  7394  16597    2750   5920   36113   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-09-04         9870     5888  ...    7358   2127     9150     2160   
2022-09-05         9870     5888  ...    7358   2127     9150     2160   
2022-09-06         9870     5888  ...    7370   2131     9164     2166   

Province_State                                                     
Admin2         Sweetwater  Teton Uinta Unassigned Washakie Weston  
2022-09-04          12063  11526  6061          0     2631   1790  
2022-09-05          12063  11526  6061          0     2631   1790  
2022-09-06          12098  11549  6066          0     2640   1807  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-09-04,20160,1338,34,31114,11933,94973,13177,11269,3070,0,...,27487,90362,4981,707,122,21463,14066,7294,15091,1881
2022-09-05,20160,1338,34,31114,11933,94995,13182,11269,3070,0,...,27487,90362,4981,707,122,21479,14066,7294,15097,1881
2022-09-06,20160,1338,34,31114,11936,95072,13182,11269,3070,0,...,27521,90415,4981,707,122,21499,14066,7301,15097,1884


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-09-04         223     699     101  105    253      54    129     644   
2022-09-05         223     699     101  105    253      54    129     644   
2022-09-06         223     699     101  105    253      54    129     644   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-09-04          167       87  ...     149     43       81       27   
2022-09-05          167       87  ...     149     43       81       27   
2022-09-06          167       87  ...     149     43       82       28   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-09-04            130    16    40          0       47     20  
2022-09-05            130    16    40          0       47     20  
2022-09-06            130    16    40          0       47     21  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change().round(7)
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-04,0.001294,0.000397,0.000063,0.0,0.0,0.0,0.0,0.001202,0.000000,0.000612,...,0.000000,0.000000,0.000000,0.000000,0.000122,0.000000,0.0,0.000419,0.000757,0.000031
2022-09-05,0.000989,0.000085,0.000067,0.0,0.0,0.0,0.0,0.000000,0.005361,0.001024,...,0.002069,0.000135,0.002366,0.000144,0.000189,0.000000,0.0,0.000000,0.000036,0.000023
2022-09-06,0.001333,0.000188,0.000056,0.0,0.0,0.0,0.0,0.000000,0.000000,0.001170,...,0.000000,0.000066,0.002360,0.000315,0.000324,0.000369,0.0,0.000000,0.000114,0.000051


In [60]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [61]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change().round(7)
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-04,0.000257,0.0,0.000145,0.0,0.0,NaN,0.0,0.000447,0.000000,0.000711,...,0.000000,0.0,0.0,0.000000,0.000023,0.000000,NaN,0.0,0.000249,0.0
2022-09-05,0.000129,0.0,0.000000,0.0,0.0,NaN,0.0,0.000000,0.000809,0.004831,...,0.001343,0.0,0.0,0.000517,0.000046,0.000000,NaN,0.0,0.000000,0.0
2022-09-06,0.000000,0.0,0.000000,0.0,0.0,NaN,0.0,0.000000,0.000000,0.004878,...,0.000000,0.0,0.0,0.000345,0.000023,0.000351,NaN,0.0,0.000000,0.0


In [62]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [63]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change().round(7)
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [65]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change().round(7)
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-09-04,0.0,0.0,0.0,0.0,0.000434,0.000000,0.000000,0.000327,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2022-09-05,0.0,0.0,0.0,0.0,0.000551,0.000451,0.001012,0.000265,0.000000,0.0,...,0.000000,0.000020,0.0,0.0,0.000000,0.002653,0.0,0.000000,0.001467,0.000000
2022-09-06,0.0,0.0,0.0,0.0,0.000458,0.000197,0.000189,0.000240,0.001953,0.0,...,0.003058,0.002805,0.0,0.0,0.003206,0.000775,0.0,0.004383,0.000360,0.002643


In [66]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [67]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change().round(7)
jh_hist_uscounties_cases_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-09-04         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-09-05         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-09-06         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ...   Wyoming                               \
Admin2         Chambers Cherokee  ...      Park    Platte Sheridan  Sublette   
2022-09-04          0.0      0.0  ...  0.000000  0.000000  0.00000  0.000000   
2022-09-05          0.0      0.0  ...  0.000000  0.000000  0.00000  0.000000   
2022-09-06          0.0      0.0  ...  0.001631  0.001881  0.00153  0.002778   

Province_State                                                                
Admin2         Sweetwater     Teton     Uinta Unassigned  Washakie    Weston  
2022-09-04       0.000000  0.000000  0.000000        NaN  0.000000  0.000000  
2022-09-05       0.000000  0.000000  0.000000        NaN  0.000000  0.000000  
2022-09-06       0.002901  0.001995  0.000825        NaN  0.003421  0.009497  

[3 rows x 3336 columns]

In [68]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [69]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change().round(7)
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-09-04,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,NaN,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.000000
2022-09-05,0.0,0.0,0.0,0.0,0.000000,0.000232,0.000379,0.0,0.0,NaN,...,0.000000,0.000000,0.0,0.0,0.0,0.000745,0.0,0.00000,0.000398,0.000000
2022-09-06,0.0,0.0,0.0,0.0,0.000251,0.000811,0.000000,0.0,0.0,NaN,...,0.001237,0.000586,0.0,0.0,0.0,0.000931,0.0,0.00096,0.000000,0.001595


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change().round(7)
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-09-04         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-09-05         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-09-06         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ... Wyoming                             \
Admin2         Chambers Cherokee  ...    Park Platte  Sheridan  Sublette   
2022-09-04          0.0      0.0  ...     0.0    0.0  0.000000  0.000000   
2022-09-05          0.0      0.0  ...     0.0    0.0  0.000000  0.000000   
2022-09-06          0.0      0.0  ...     0.0    0.0  0.012346  0.037037   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-09-04            0.0   0.0   0.0        NaN      0.0   0.00  
2022-09-05            0.0   0.0   0.0        NaN      0.0   0.00  
2022-09-06            0.0   0.0   0.0        NaN      0.0   0.05  

[3 rows x 3336 columns]

In [72]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [73]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean().round(7)
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-04,0.001319,0.000555,0.000093,3.000000e-07,0.0,0.0,0.000045,0.000609,0.000039,0.000699,...,0.000015,0.000016,0.000234,0.000184,0.000142,0.000102,0.0,0.000212,0.000382,0.000035
2022-09-05,0.001154,0.000320,0.000080,1.000000e-07,0.0,0.0,0.000023,0.000305,0.002700,0.000862,...,0.001042,0.000076,0.001300,0.000164,0.000166,0.000051,0.0,0.000106,0.000209,0.000029
2022-09-06,0.001243,0.000254,0.000068,1.000000e-07,0.0,0.0,0.000011,0.000152,0.001350,0.001016,...,0.000521,0.000071,0.001830,0.000239,0.000245,0.000210,0.0,0.000053,0.000161,0.000040


In [74]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [75]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-04,0.000204,0.000023,0.000073,0.0,0.0,NaN,1.700000e-06,0.000226,0.000006,0.001527,...,0.000007,0.0,0.0,0.000088,0.000022,0.000038,NaN,1.000000e-07,0.000124,0.000017
2022-09-05,0.000166,0.000011,0.000036,0.0,0.0,NaN,8.000000e-07,0.000113,0.000408,0.003179,...,0.000675,0.0,0.0,0.000303,0.000034,0.000019,NaN,1.000000e-07,0.000062,0.000008
2022-09-06,0.000083,0.000006,0.000018,0.0,0.0,NaN,4.000000e-07,0.000056,0.000204,0.004028,...,0.000338,0.0,0.0,0.000324,0.000029,0.000185,NaN,0.000000e+00,0.000031,0.000004


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [77]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean().round(7)
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-04,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-09-05,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-09-06,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078


In [78]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [79]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-09-04,0.000626,0.000153,0.000008,0.000121,0.000762,0.000356,0.000218,0.000805,0.000184,0.0,...,0.000306,0.000326,0.000181,0.000121,0.000353,0.000327,0.000226,0.000445,0.000211,0.000056
2022-09-05,0.000313,0.000077,0.000004,0.000061,0.000656,0.000404,0.000615,0.000535,0.000092,0.0,...,0.000153,0.000173,0.000090,0.000061,0.000176,0.001490,0.000113,0.000223,0.000839,0.000028
2022-09-06,0.000157,0.000038,0.000002,0.000030,0.000557,0.000300,0.000402,0.000388,0.001022,0.0,...,0.001605,0.001489,0.000045,0.000030,0.001691,0.001133,0.000057,0.002303,0.000600,0.001335


In [80]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [81]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-09-04      0.000469  0.000567  0.001013  0.000892  0.000387  0.000812   
2022-09-05      0.000235  0.000284  0.000507  0.000446  0.000193  0.000406   
2022-09-06      0.000117  0.000142  0.000253  0.000223  0.000097  0.000203   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-09-04      0.000830  0.000662  0.000451  0.000900  ...  0.000045   
2022-09-05      0.000415  0.000331  0.000226  0.000450  ...  0.000023   
2022-09-06      0.000207  0.000166  0.000113  0.000225  ...  0.000827   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-09-04      0.000045  0.000057  0.000008   0.000051  0.000059  0.000041   
2022-09-05      0.000022  0.000029  0.000004   0.000026  0.000029  0.000021   
2022-09-06      0.000951  0.000779  0.001391   0.001463  0.001012  0.000423   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2022-09-04      -0.503937  0.000090  0.000063  
2022-09-05      -0.503937  0.000045  0.000031  
2022-09-06      -0.503937  0.001733  0.004764  

[3 rows x 3336 columns]

In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [83]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-09-04,0.000349,1.500000e-06,0.000015,0.000068,0.000204,0.000117,0.000106,0.000115,0.001151,NaN,...,0.000134,0.000094,0.000165,1.700000e-06,0.0,0.000262,0.000311,0.000147,0.000140,0.000034
2022-09-05,0.000175,7.000000e-07,0.000007,0.000034,0.000102,0.000175,0.000243,0.000057,0.000575,NaN,...,0.000067,0.000047,0.000082,9.000000e-07,0.0,0.000504,0.000155,0.000074,0.000269,0.000017
2022-09-06,0.000087,4.000000e-07,0.000004,0.000017,0.000177,0.000492,0.000121,0.000029,0.000288,NaN,...,0.000652,0.000317,0.000041,4.000000e-07,0.0,0.000717,0.000078,0.000517,0.000134,0.000806


In [84]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [85]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin Barbour Bibb    Blount Bullock Butler   
2022-09-04      0.000282  0.000541     0.0  0.0  0.000498     0.0    0.0   
2022-09-05      0.000141  0.000271     0.0  0.0  0.000249     0.0    0.0   
2022-09-06      0.000070  0.000135     0.0  0.0  0.000124     0.0    0.0   

Province_State                               ... Wyoming                      \
Admin2           Calhoun  Chambers Cherokee  ...    Park    Platte  Sheridan   
2022-09-04      0.000391  0.000758      0.0  ...     0.0  0.000372 -0.000000   
2022-09-05      0.000195  0.000379      0.0  ...     0.0  0.000186 -0.000000   
2022-09-06      0.000098  0.000189      0.0  ...     0.0  0.000093  0.006173   

Province_State                                                              
Admin2          Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-09-04      0.000000   0.000121   0.0   0.0       -1.0      0.0  0.000  
2022-09-05      0.000000   0.000061   0.0   0.0       -1.0      0.0  0.000  
2022-09-06      0.018519   0.000030   0.0   0.0       -1.0      0.0  0.025  

[3 rows x 3336 columns]

In [86]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [87]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

HTTPError: HTTP Error 403: Forbidden

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [ ]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [ ]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

In [ ]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

#### 